In [1]:
Pkg.add("MNIST")

INFO: Package MNIST is already installed
INFO: METADATA is out-of-date — you may not have the latest version of MNIST
INFO: Use `Pkg.update()` to get the latest versions of your packages


In [2]:
using MNIST
trainX, trainY = traindata()
testX, testY = testdata()
trainY=Int.(trainY)
testY=Int.(testY)

Stacktrace:
 [1] depwarn(::String, ::Symbol) at .\deprecated.jl:70
 [2] Array(::Type{Float64}, ::Int64, ::Int64) at .\deprecated.jl:57
 [3] traindata() at C:\Users\User\AppData\Local\JuliaPro-0.6.0.1\pkgs-0.6.0.1\v0.6\MNIST\src\MNIST.jl:88
 [4] include_string(::String, ::String) at .\loading.jl:515
 [5] execute_request(::ZMQ.Socket, ::IJulia.Msg) at C:\Users\User\AppData\Local\JuliaPro-0.6.0.1\pkgs-0.6.0.1\v0.6\IJulia\src\execute_request.jl:160
 [6] eventloop(::ZMQ.Socket) at C:\Users\User\AppData\Local\JuliaPro-0.6.0.1\pkgs-0.6.0.1\v0.6\IJulia\src\eventloop.jl:8
 [7] (::IJulia.##11#14)() at .\task.jl:335
while loading In[2], in expression starting on line 2
Stacktrace:
 [1] depwarn(::String, ::Symbol) at .\deprecated.jl:70
 [2] Array(::Type{Float64}, ::Int64) at .\deprecated.jl:57
 [3] traindata() at C:\Users\User\AppData\Local\JuliaPro-0.6.0.1\pkgs-0.6.0.1\v0.6\MNIST\src\MNIST.jl:89
 [4] include_string(::String, ::String) at .\loading.jl:515
 [5] execute_request(::ZMQ.Socket, ::IJuli

10000-element Array{Int64,1}:
 7
 2
 1
 0
 4
 1
 4
 9
 5
 9
 0
 6
 9
 ⋮
 5
 6
 7
 8
 9
 0
 1
 2
 3
 4
 5
 6

In [3]:
type NeuralNetwork
    numOfLayers::Int
    sizes::Array{Int,1}
    biases::Array{Array{Float64,1},1}
    weights::Array{Array{Float64,2},1}
    function NeuralNetwork(s)
        numOfLayers=length(s)
        sizes=s
        biases=[rand(y) for y in s[2:end]]
        weights=[rand(y,x) for (x,y) in zip(s[1:end-1],s[2:end])]
        new(numOfLayers,sizes,biases,weights)
    #return biases, weights
    end
end

In [4]:
net = NeuralNetwork([784,15,10])

NeuralNetwork(3, [784, 15, 10], Array{Float64,1}[[0.513886, 0.000423805, 0.962601, 0.169844, 0.389471, 0.676696, 0.677257, 0.149039, 0.126735, 0.242021, 0.501019, 0.20059, 0.285737, 0.695962, 0.244196], [0.454473, 0.484908, 0.90145, 0.120237, 0.131669, 0.592903, 0.209748, 0.593931, 0.482784, 0.909551]], Array{Float64,2}[[0.138632 0.503996 … 0.00588135 0.248403; 0.716476 0.113758 … 0.354183 0.370511; … ; 0.401057 0.688614 … 0.42302 0.609837; 0.304336 0.124245 … 0.916517 0.94341], [0.234465 0.923546 … 0.203764 0.878007; 0.304044 0.821021 … 0.557493 0.217041; … ; 0.786513 0.56555 … 0.134947 0.172172; 0.0559474 0.833103 … 0.286756 0.301428]])

In [5]:
function sigmoid(z)
  return 1.0 / (1.0 + exp(-z))
end


sigmoid (generic function with 1 method)

In [6]:
function feedforward(net,a)
    for (b,w) in zip(net.biases[1:end-1], net.weights[1:end-1])
        a = sigmoid.((w*a)+b)
    end
   #d is the sum over all output neurons
    d=0
    for k in a
        d+=exp(k)
    end
    for (b,w) in zip(net.biases[end], net.weights[end])
        a = softmax.((w*a)+b,d)
    end
    #println(sum(a))
    return a
end

feedforward (generic function with 1 method)

In [7]:
function softmax(z,d) 
    return exp(z)/d
end

softmax (generic function with 1 method)

In [8]:
function SGD(net, training_data, epochs, mini_batch_size, eta,test_data)
    n_test = length(test_data)
    n = length(training_data)
    training_data2=collect(training_data)
    mini_batches = [training_data2[k:k+(mini_batch_size-1)] for k=1:mini_batch_size:n-1]
    for j in 1:epochs
        for mini_batch in mini_batches
            update_mini_batch(net, mini_batch, eta)
        end
       
        print("epoch ",j, ": ",evaluate(net,test_data),"  ")
        
    end
end

SGD (generic function with 1 method)

In [9]:
function update_mini_batch(net, mini_batch, eta)
    #def fn(a, y):
     #   return np.sum(np.nan_to_num(-y*log(a)-(1-y)*log(1-a)))
    nabla_b = [zeros(size(b)) for b in net.biases]
    nabla_w = [zeros(size(w)) for w in net.weights]
    for (x, y) in mini_batch
        delta_nabla_b, delta_nabla_w = backprop(net,x, y)
        nabla_b = [nb+dnb for (nb, dnb) in zip(nabla_b, delta_nabla_b)]
        nabla_w = [nw+dnw for (nw, dnw) in zip(nabla_w, delta_nabla_w)]
    end
    net.weights = [w-(eta/size(mini_batch,1))*nw for (w, nw) in zip(net.weights, nabla_w)]
    net.biases = [b-(eta/size(mini_batch,1))*nb for (b, nb) in zip(net.biases, nabla_b)]
    
    return net.weights,net.biases
end

update_mini_batch (generic function with 1 method)

In [10]:
function backprop(net,x, y)
    nabla_b = [zeros(size(b)) for b in net.biases]
    nabla_w = [zeros(size(w)) for w in net.weights]
    # feedforward
    activation = x
    activations = [x] # list to store all the activations, layer by layer
    zs = [] # list to store all the z vectors, layer by layer
    for (b, w) in zip(net.biases, net.weights)
        #z = dot(w, activation)+b
        z = (w*activation)+b
        push!(zs,z)
        activation = sigmoid.(z)
        push!(activations,activation)
    end
    # backward pass
    delta1 = Array(cost_derivative(net,activations[end], y))
    delta=[]
    for i in delta1
        push!(delta,i)
    end
    nabla_b[end] = delta
    nabla_w[end] = delta*transpose(activations[end-1])
    # Note that the variable l in the loop below is used a little
    # differently to the notation in Chapter 2 of the book.  Here,
    # l = 1 means the last layer of neurons, l = 2 is the
    # second-last layer, and so on.  It's a renumbering of the
    # scheme in the book, used here to take advantage of the fact
    # that Python can use negative indices in lists.
    for l=net.numOfLayers-1:-1:2
        z = zs[l-1]
        #println(size(net.weights[l]), " ", size(delta)," ", size(sp))
        delta = (transpose(net.weights[l])*delta) 
        nabla_b[l-1] = delta
        nabla_w[l-1] = (delta * transpose(activations[l-1]))
    end
    return nabla_b, nabla_w
end

backprop (generic function with 1 method)

In [11]:
function cross_entropy(a,y)
     return (-y*log(a)-(1-y)*log(1-a))
end

cross_entropy (generic function with 1 method)

In [12]:
function evaluate(net,test_data)
    test_results = [(findmax(feedforward(net,x))[2], findmax(y)[2]) for (x, y) in test_data]
    #println(test_results)
    return  sum((x == y) for (x, y) in test_results) / size(test_data)[1] 
end

evaluate (generic function with 1 method)

In [13]:
function cost_derivative(net,output_activations, y)
    return output_activations-y
end

cost_derivative (generic function with 1 method)

In [14]:
function vectorized_result(j)
    e = zeros(10, 1)
    e[j+1] = 1.0
    return e
end

vectorized_result (generic function with 1 method)

In [15]:
training_inp=reshape(trainX,(60000,784))
training_inputs=[]
for i=1:size(training_inp)[1]
    push!(training_inputs,training_inp[i,:])
end
training_outputs = [vectorized_result(y) for y in trainY]
training_data = zip(training_inputs, training_outputs)

test_inp=reshape(testX,(10000,784))
test_inputs=[]
for i=1:size(test_inp)[1]
    push!(test_inputs,test_inp[i,:])
end
test_outputs = [vectorized_result(y) for y in testY]
test_data = zip(test_inputs, test_outputs)


Base.Iterators.Zip2{Array{Any,1},Array{Array{Float64,2},1}}(Any[[0.0, 0.0, 73.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 194.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 249.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 208.0, 0.0], [0.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  41.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 16.0, 0.0], [0.0, 92.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  132.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 67.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  213.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  254.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  253.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 254.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 1

In [ ]:
SGD(net,training_data, 100 , 30 , 0.5, test_data)

epoch 1: 0.098  epoch 2: 0.098  epoch 3: 0.098  epoch 4: 0.098  

#### Note I already tried running 30 epochs and network was stuck at the same accuracy but the results were not saved when I closed the notebook